In [ ]:
# default_exp blocker
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


# Website Blocker

> This is the core functionality of beproductive. A defined list of websites is added to the `hosts` file of the OS with a redirect to `127.0.0.1`.

In [ ]:
#hide
from nbdev.showdoc import *

In [ ]:
#export
from pathlib import Path
from shutil import copy
import sys
try:
    from win10toast import ToastNotifier
    win_notify = ToastNotifier()
except:
    win_notify = False

In [ ]:
#export
APP_NAME = 'Website Blocker'
REDIRECT = '127.0.0.1'
WIN_PATH = r'C:\Windows\System32\drivers\etc'
LINUX_PATH = r'/etc'
ICON_PATH = 'icon.ico'
BLOCKLIST = 'blocklist.txt'

if sys.platform == 'win32':
    host_path = Path(WIN_PATH)
else:
    host_path = Path(LINUX_PATH)
host_fp = host_path/'hosts'
host_fp_copy = host_path/'hosts.original'
host_fp_blocked = host_path/'hosts.blocked'

In [ ]:
#export
class Blocker():
    "The core of the package. It modifies the hosts file of the OS."
    def __init__(self, redirect=REDIRECT):
        self.adminrights = False
        self.redirect = redirect
        with open(BLOCKLIST) as file:
            self.blocked = [line.rstrip() for line in file]
        if not host_fp_copy.exists():
            self._setup()
        if self._create_blocked_list():
            self.adminrights = True

    def _setup(self):
        "Creates a copy of the `hosts` file and saves it as `hosts.original`"
        try:
            copy(host_fp, host_fp_copy)
            self.notify("Setup successful")
        except PermissionError:
            self._raise_permission_error()

    def _create_blocked_list(self):
        "Creates a copy of `hosts.original` and saves it to `hosts.blocked`. Then adds all blocked sites."
        try:
            copy(host_fp_copy, host_fp_blocked)
            with open(host_fp_blocked, "a") as blocked_file:
                for url in self.blocked:
                    # TODO: refine, add www only if not in url
                    blocked_file.write(f"{self.redirect} {url}\n")
                    blocked_file.write(f"{self.redirect} www.{url}\n")
            return True
        except PermissionError:
            self._raise_permission_error()
            return False

    def block(self):
        "Blocks all specified websites by replacing `hosts` file with `hosts.blocked`"
        try:
            copy(host_fp_blocked, host_fp)
            return True
        except PermissionError:
            self._raise_permission_error()
            return False

    def unblock(self):
        "Unblocks all websites by restoring the original `hosts` file"
        try:
            copy(host_fp_copy, host_fp)
            return True
        except PermissionError:
            self._raise_permission_error()
            return False

    def notify(self, message, title=APP_NAME, duration=5):
        "Sends notification to CLI and - if available - to GUI"
        print(message)
        if win_notify:
            win_notify.show_toast(title, message, duration=duration) # icon_path=ICON_PATH, 

    def _raise_permission_error(self):
        self.notify("Permission Error. Please run the command line tool as ADMINISTRATOR.")

In [ ]:
show_doc(Blocker.block)

<h4 id="Blocker.block" class="doc_header"><code>Blocker.block</code><a href="__main__.py#L36" class="source_link" style="float:right">[source]</a></h4>

> <code>Blocker.block</code>()

Blocks all specified websites by replacing `hosts` file with `hosts.blocked`

In [ ]:
show_doc(Blocker.unblock)

<h4 id="Blocker.unblock" class="doc_header"><code>Blocker.unblock</code><a href="__main__.py#L45" class="source_link" style="float:right">[source]</a></h4>

> <code>Blocker.unblock</code>()

Unblocks all websites by restoring the original `hosts` file

In [ ]:
show_doc(Blocker.notify)

<h4 id="Blocker.notify" class="doc_header"><code>Blocker.notify</code><a href="__main__.py#L54" class="source_link" style="float:right">[source]</a></h4>

> <code>Blocker.notify</code>(**`message`**, **`title`**=*`'Website Blocker'`*, **`duration`**=*`5`*)

Sends notification to CLI and - if available - to GUI

## Using Blocker

In [ ]:
blocker = Blocker()

In [ ]:
with open(host_fp) as host_file:
    with open(host_fp_copy) as host_original:
        assert host_file.readlines() == host_original.readlines(), "hosts file should be identical to hosts.original"

In [ ]:
blocker.block()

True

In [ ]:
with open(host_fp) as host_file:
    with open(host_fp_blocked) as host_blocked:
        assert host_file.readlines() == host_blocked.readlines(), "hosts file should be identical to hosts.blocked"

In [ ]:
blocker.unblock()

True

In [ ]:
with open(host_fp) as host_file:
    with open(host_fp_copy) as host_original:
        assert host_file.readlines() == host_original.readlines(), "hosts file should be identical to hosts.original"

## Command line tool

In [ ]:
#export
if __name__=='__main__':
    blocker = Blocker()
    if blocker.adminrights:
        try: mode = sys.argv[1].lower()
        except: mode = 'none' 

        if mode == 'block':
            if blocker.block():
                blocker.notify("Websites blocked, enjoy your work")
        elif mode == 'unblock':
            if blocker.unblock():
                blocker.notify("Websites unblocked, have fun")
        else:
            print("Run blocker.py with arguments BLOCK or UNBLOCK")

Run blocker.py with arguments BLOCK or UNBLOCK


In [ ]:
from nbdev.export import notebook2script; notebook2script()

Converted 00_blocker.ipynb.
Converted 01_pomodoro.ipynb.
Converted index.ipynb.
